**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [20]:
!pip install scikit-video
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd,Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

You should consider upgrading via the 'pip install --upgrade pip' command.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act gives the next action the agent does on the next step on the current states (via the function learned_act) or on a random basis (with some probability if there is training). 

The parameter $\epsilon$ controls the probability to act either on a random basis or with the learned action. If $\epsilon$ is low, we just act with respect to what we have learned. And if it is high, we only act randomly.

This algorithm called $\epsilon$-greedy tries to output the best decision according to current information (exploitation of data) and in the meantime, gathers more information (exploring). $\epsilon$ is the parameter dealing with both exploitation and exploration.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [26]:
# parameters
size = 13
T=200
temperature=0.3 #0.3
epochs_train=20 # set small when debugging
epochs_test=20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The position array gives the grid of possible/current positions of the rat for $(x,y)$:

   -position[x,y] = -1 if the rat cannot go on the cell $(x,y)$

   -position[x,y] = 0 if the rat can go on the cell $(x,y)$ 

   -position[x,y] = 1 if the rat can go on the cell $(x,y)$ 

The board array gives the reward for each cell $(x,y)$ : if the rat goes there, it gets board[x,y].

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [10]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [11]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE # taken from description above
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False
        win = 0
        lose = 0
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [12]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
###### mplyer used to show the videos directly
#HTML(display_videos('random10.mp4'))
!mplayer 'random10.mp4'

Win/lose count 7.5/12.0. Average score (-4.5)
Win/lose count 7.5/8.0. Average score (-2.5)
Win/lose count 9.5/11.0. Average score (-2.1666666666666665)
Win/lose count 9.0/22.0. Average score (-4.875)
Win/lose count 11.0/15.0. Average score (-4.7)
Win/lose count 11.5/10.0. Average score (-3.6666666666666665)
Win/lose count 7.5/20.0. Average score (-4.928571428571429)
Win/lose count 11.0/14.0. Average score (-4.6875)
Win/lose count 7.5/12.0. Average score (-4.666666666666667)
Win/lose count 8.0/9.0. Average score (-4.3)
Win/lose count 6.5/15.0. Average score (-4.681818181818182)
Win/lose count 10.5/19.0. Average score (-5.0)
Win/lose count 7.5/22.0. Average score (-5.730769230769231)
Win/lose count 6.5/18.0. Average score (-6.142857142857143)
Win/lose count 11.5/19.0. Average score (-6.233333333333333)
Win/lose count 9.0/11.0. Average score (-5.96875)
Win/lose count 5.5/14.0. Average score (-6.117647058823529)
Win/lose count 10.5/12.0. Average score (-5.861111111111111)
Win/lose count 6.

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




## Answer :

### First equation :
We have :

$$Q^{\pi}(s,a)=E[ \sum_{k=0}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] = r(s,a) + E[\sum_{k=1}^{\infty} \gamma^k r_{t+k}|s_t = s, a_t = a] $$
Then we condition on some other states and action :
$$ = r(s,a) + \gamma E[E[\sum_{k=0}^{\infty}[\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']|s_t = s, a_t = a] $$
We can now use conditioned probabilities to express $E$ and as the probabilities other all sates and actions sum to 1, we deduce:
$$ = \sum_{(s',a')} (p(s_{t+1}=s',a_{t+1}=a' |s_t = s, a_t = a ) [r(s,a)+\gamma E[\sum_{k=0}^{\infty}\gamma^k r_{t+1+k}|s_{t+1}=s',a_{t+1}=a']]$$
$$=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] $$

### Second equation :
With the above expression and by optimility of the policy $\pi^*$, for a fixed state $s^*$, $Q^{\pi^{*}}(s, a)=\max _{a'} Q^{\pi^{*}} (s, a')$. We note $a_{\pi^{*}}(s)$ that maximises the quantity.

Then 
$$Q^{*}(s,a) = \max_{\pi}E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] $$
$$ = E_{(s')\sim \pi^*(.|s,a)} [r(s,a)+\gamma Q^{\pi^*}(s',a_{\pi^{*}}(s'))] $$
$$ = E_{(s')\sim \pi^*(.|s,a)} [r(s,a)+\gamma \max_{a'} Q^{*}(s',a')] $$

### Third equation :

If $Q$ is parametrized by $\theta$, $||Q^{*}(s,a)-Q_{\theta}(𝑠,𝑎)||^2$ will be the squared loss to minimize. We then approximate $Q^{*}(s,a)$ by the above expression that we estimate at each step : $E_{(s')\sim \pi^*(.|s,a)} [r(s,a)+\gamma \max_{a'} Q_{\theta}(s',a')]$. We then have to consider the loss term : $E_{(s')\sim \pi^*(.|s,a)} [r(s,a)+\gamma \max_{a'} Q_{\theta}(s',a')]-Q_{\theta}(𝑠,𝑎)$. As $Q(s,a)$ is a constant for the expectancy, so a plausible loss function will be : $L(\theta)=E_{(s')\sim \pi^*(.|s,a)} ||r(s,a)+\gamma max_{a'}Q(s',a',\theta)-Q(s,a, \theta) ||^2$.







***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [13]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        # if overflow : remove the first element
        if len(self.memory) > self.max_memory:
            self.memory = self.memory[1:]

    def random_access(self):
        return self.memory[np.random.randint(0,len(self.memory), size=1)[0]]

***
The pipeline we will use for training is given below:

In [14]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [23]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.05, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99 
        self.grid_size = grid_size        
        # number of state
        self.n_state = n_state
        # Memory
        self.memory = Memory(memory_size)        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape([1,s.shape[0],s.shape[1],s.shape[2]]))[0,:])
        #pass

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))

        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_  = self.memory.random_access()   
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
            if game_over_:
                ######## FILL IN
                input_states[i] = s_
                target_q[i,a_] = r_
            else:
                ######## FILL IN
                input_states[i] = s_
                target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0])
        ######## FILL IN
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(32, activation='relu'))
        model.add(Dense(4, activation='linear'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        #model.compile(loss='mse', optimizer=Adam(lr=lr))
        self.model = model
        

In [25]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
#HTML(display_videos('fc_train10.mp4'))
!mplayer './fc_train10.mp4'

Epoch 000/050 | Loss 0.0084 | Win/lose count 3.5/6.0 (-2.5)
Epoch 001/050 | Loss 0.0018 | Win/lose count 2.5/3.0 (-0.5)
Epoch 002/050 | Loss 0.0021 | Win/lose count 1.5/5.0 (-3.5)
Epoch 003/050 | Loss 0.0034 | Win/lose count 5.0/6.0 (-1.0)
Epoch 004/050 | Loss 0.0219 | Win/lose count 7.5/6.0 (1.5)
Epoch 005/050 | Loss 0.0091 | Win/lose count 5.0/7.0 (-2.0)
Epoch 006/050 | Loss 0.0078 | Win/lose count 2.5/7.0 (-4.5)
Epoch 007/050 | Loss 0.0070 | Win/lose count 2.5/3.0 (-0.5)
Epoch 008/050 | Loss 0.0085 | Win/lose count 3.5/6.0 (-2.5)
Epoch 009/050 | Loss 0.0139 | Win/lose count 4.5/6.0 (-1.5)
Epoch 010/050 | Loss 0.0083 | Win/lose count 3.5/4.0 (-0.5)
Epoch 011/050 | Loss 0.0159 | Win/lose count 9.5/2.0 (7.5)
Epoch 012/050 | Loss 0.0156 | Win/lose count 1.0/2.0 (-1.0)
Epoch 013/050 | Loss 0.0128 | Win/lose count 5.5/7.0 (-1.5)
Epoch 014/050 | Loss 0.0550 | Win/lose count 8.0/3.0 (5.0)
Epoch 015/050 | Loss 0.0091 | Win/lose count 5.0/3.0 (2.0)
Epoch 016/050 | Loss 0.0115 | Win/lose count

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [19]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(filters=32, kernel_size=2, activation="relu", padding="valid", input_shape=(5, 5, self.n_state)))
        model.add(Conv2D(filters=16, kernel_size=2, activation="relu"))
        model.add(Flatten())
        model.add(Dense(4, activation=None))
        
        #model.compile(loss='mse', optimizer=Adam(lr=lr))
        #self.model = model
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [22]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
#HTML(display_videos('cnn_train10.mp4'))
!mplayer 'cnn_train10.mp4'

Epoch 000/005 | Loss 0.0041 | Win/lose count 2.5/2.0 (0.5)
Epoch 001/005 | Loss 0.0112 | Win/lose count 2.0/1.0 (1.0)
Epoch 002/005 | Loss 0.0006 | Win/lose count 2.0/2.0 (0.0)
Epoch 003/005 | Loss 0.0115 | Win/lose count 2.0/3.0 (-1.0)
Epoch 004/005 | Loss 0.0091 | Win/lose count 1.0/4.0 (-3.0)
MPlayer 1.3.0 (Debian), built with gcc-7 (C) 2000-2016 MPlayer Team
do_connect: could not connect to socket
connect: No such file or directory
Failed to open LIRC support. You will not be able to use your remote control.

Playing cnn_train0.mp4.
libavformat version 57.83.100 (external)
libavformat file format detected.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7f2254b0b2a0]Protocol name not provided, cannot determine if input is local or a network protocol, buffers and access patterns cannot be configured optimally without knowing the protocol
[lavf] stream 0: video (h264), -vid 0
VIDEO:  [H264]  272x272  24bpp  25.000 fps    5.1 kbps ( 0.6 kbyte/s)
Failed to open VDPAU backend libvdpau_va_gl.so: cannot ope

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [32]:
env = Environment(grid_size=size, max_time=T,temperature=10)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 2.0/4.0. Average score (-2.0)
Win/lose count 2.5/3.0. Average score (-1.25)
Win/lose count 0/3.0. Average score (-1.8333333333333333)
Win/lose count 3.0/4.0. Average score (-1.625)
Win/lose count 1.5/2.0. Average score (-1.4)
Win/lose count 1.0/2.0. Average score (-1.3333333333333333)
Win/lose count 1.5/1.0. Average score (-1.0714285714285714)
Win/lose count 1.0/2.0. Average score (-1.0625)
Win/lose count 2.0/1.0. Average score (-0.8333333333333334)
Win/lose count 2.0/2.0. Average score (-0.75)
Win/lose count 0.5/0. Average score (-0.6363636363636364)
Win/lose count 3.0/1.0. Average score (-0.4166666666666667)
Win/lose count 1.5/5.0. Average score (-0.6538461538461539)
Win/lose count 1.5/2.0. Average score (-0.6428571428571429)
Win/lose count 1.5/3.0. Average score (-0.7)
Win/lose count 2.0/2.0. Average score (-0.65625)
Win/lose count 2.5/1.0. Average score (-0.5294117647058824)
Win/lose count 3.5/2.0. Average score (-0.4166666666666667)
Win/lose count 2.

In [33]:
#HTML(display_videos('cnn_test10.mp4'))
!mplayer 'cnn_test10.mp4'

MPlayer 1.3.0 (Debian), built with gcc-7 (C) 2000-2016 MPlayer Team
do_connect: could not connect to socket
connect: No such file or directory
Failed to open LIRC support. You will not be able to use your remote control.

Playing cnn_test19.mp4.
libavformat version 57.83.100 (external)
libavformat file format detected.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7f9d5fa192a0]Protocol name not provided, cannot determine if input is local or a network protocol, buffers and access patterns cannot be configured optimally without knowing the protocol
[lavf] stream 0: video (h264), -vid 0
VIDEO:  [H264]  272x272  24bpp  25.000 fps    5.1 kbps ( 0.6 kbyte/s)
Failed to open VDPAU backend libvdpau_va_gl.so: cannot open shared object file: No such file or directory
[vdpau] Error when calling vdp_device_create_x11: 1
Opening video decoder: [ffmpeg] FFmpeg's libavcodec codec family
libavcodec version 57.107.100 (external)
Selected video codec: [ffh264] vfm: ffmpeg (FFmpeg H.264)
Clip info:
 major_brand: isom
 mi

In [34]:
#HTML(display_videos('fc_test10.mp4'))
!mplayer 'fc_test10.mp4'

MPlayer 1.3.0 (Debian), built with gcc-7 (C) 2000-2016 MPlayer Team
do_connect: could not connect to socket
connect: No such file or directory
Failed to open LIRC support. You will not be able to use your remote control.

Playing fc_test19.mp4.
libavformat version 57.83.100 (external)
libavformat file format detected.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7fc79c9032a0]Protocol name not provided, cannot determine if input is local or a network protocol, buffers and access patterns cannot be configured optimally without knowing the protocol
[lavf] stream 0: video (h264), -vid 0
VIDEO:  [H264]  272x272  24bpp  25.000 fps    5.0 kbps ( 0.6 kbyte/s)
Failed to open VDPAU backend libvdpau_va_gl.so: cannot open shared object file: No such file or directory
[vdpau] Error when calling vdp_device_create_x11: 1
Opening video decoder: [ffmpeg] FFmpeg's libavcodec codec family
libavcodec version 57.107.100 (external)
Selected video codec: [ffh264] vfm: ffmpeg (FFmpeg H.264)
Clip info:
 major_brand: isom
 min

### Interpretation:

The CNN seems better in terms of reward and exploration of the board. 

If we change the temperature, for 0.3, the CNN gives a reward of -0.225 whereas the FC gives -0.25; if we increase it to 10, we have -0.45 for the CNN and -0.75 for the FC; the rat tends to move more in high temperature cases.

For both models, in the videos, we see that the rat seems to return to its steps :the exploration of the all space is not effective.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [38]:
def train_explore(agent,env,epoch,prefix=''):
    score = 0
    loss = 0
    decay = (agent.epsilon-0.002)/epoch

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        agent.set_epsilon(agent.epsilon*(1-decay))
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            # use this sample code
            state, reward, game_over = env.act(action, train=True)
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    #pass
        
class EnvironmentExploring(object):       
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        self.x = 0
        self.y = 1

        # self time
        self.t = 0
        self.scale=16
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward += -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        #self.t = self.t + 1
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                self.board.reshape(self.grid_size, self.grid_size, 1),
                self.position.reshape(self.grid_size, self.grid_size,1)), axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]     
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)
        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)
        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))
        malus[bonus>0]=0
        self.board = bonus + malus
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        return state
        #pass
        
        
## use those samples of code:
#In train explore:
#state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
#reward = 0
#if train:
#    reward = -self.malus_position[self.x, self.y]
#self.malus_position[self.x, self.y] = 0.1

#reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
#state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                self.board.reshape(self.grid_size, self.grid_size,1),
#                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [39]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))
!mplayer 'cnn_train_explore10.mp4'

Epoch 000/020 | Loss 0.1683 | Win/lose count 1.5/21.900000000000013 (-20.400000000000013)
Epoch 001/020 | Loss 0.0673 | Win/lose count 2.0/22.900000000000052 (-20.900000000000052)
Epoch 002/020 | Loss 0.0043 | Win/lose count 1.0/23.100000000000044 (-22.100000000000044)
Epoch 003/020 | Loss 0.0019 | Win/lose count 0/24.900000000000084 (-24.900000000000084)
Epoch 004/020 | Loss 0.0027 | Win/lose count 2.5/23.60000000000001 (-21.10000000000001)
Epoch 005/020 | Loss 0.1950 | Win/lose count 2.5/21.00000000000002 (-18.50000000000002)
Epoch 006/020 | Loss 0.2289 | Win/lose count 2.5/19.600000000000012 (-17.100000000000012)
Epoch 007/020 | Loss 0.0024 | Win/lose count 3.5/20.100000000000016 (-16.600000000000016)
Epoch 008/020 | Loss 0.0021 | Win/lose count 1.0/22.900000000000066 (-21.900000000000066)
Epoch 009/020 | Loss 0.0027 | Win/lose count 1.5/20.200000000000017 (-18.700000000000017)
Epoch 010/020 | Loss 0.0028 | Win/lose count 2.0/22.70000000000005 (-20.70000000000005)
Epoch 011/020 | Lo

In [40]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))
!mplayer 'cnn_test_explore10.mp4'

MPlayer 1.3.0 (Debian), built with gcc-7 (C) 2000-2016 MPlayer Team
do_connect: could not connect to socket
connect: No such file or directory
Failed to open LIRC support. You will not be able to use your remote control.

Playing cnn_train_explore10.mp4.
libavformat version 57.83.100 (external)
libavformat file format detected.
[mov,mp4,m4a,3gp,3g2,mj2 @ 0x7f98f472f2a0]Protocol name not provided, cannot determine if input is local or a network protocol, buffers and access patterns cannot be configured optimally without knowing the protocol
[lavf] stream 0: video (h264), -vid 0
VIDEO:  [H264]  272x272  24bpp  25.000 fps    5.0 kbps ( 0.6 kbyte/s)
Failed to open VDPAU backend libvdpau_va_gl.so: cannot open shared object file: No such file or directory
[vdpau] Error when calling vdp_device_create_x11: 1
Opening video decoder: [ffmpeg] FFmpeg's libavcodec codec family
libavcodec version 57.107.100 (external)
Selected video codec: [ffh264] vfm: ffmpeg (FFmpeg H.264)
Clip info:
 major_brand:

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***